In [ ]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
from os.path import join as pjoin
import csv
import itertools
from collections import Counter

### The workflow

1. prepare the matertial: 
    - stimulus: `h5py` file with size
    - ROI: `nii.gz` file, and the label: `ctab files`, with two template, `kastner2015`, and `prf-visualrois`.
    - beta values: 750 betas, each scan session, and 12 runs each session (37 session in total). 
2. finding labels for each voxel, so that I can know what regions they are in.
3. for each trial, find labels for each voxel. Dont average.
4. then save to an array using image as index, to average the trials so that each image has 3 trials.

### Reading the stimuli dataset
The size of the stimuli: (730000, 425, 425, 3), which corresponds to 730000 images of size 425x425 with 3 color channels (RGB).

In [ ]:
basedir = '/mnt/c/Users/Wayne/Desktop/nsd'
stimuli_dir = pjoin(basedir, 'nsd_stimuli')
stimuli_file = pjoin(stimuli_dir, 'nsd_stimuli.hdf5')

# read hdf5 file
with h5py.File(stimuli_file, 'r') as f:
    # get data key
    data_key = list(f.keys())[0]
    dataset = f[data_key]
    print('dataset shape: ', dataset.shape)

### Read NSD beta
There is 37 session in total. Each session has 750 trials. 

The 3D voxel space of the brain is (83, 104, 81).

In [ ]:
betas_dir = pjoin(basedir, 'nsd_betas')
betas_file = pjoin(betas_dir, 'betas_session01.hdf5')

# read hdf5 file
f = h5py.File(betas_file, 'r')
# read s
data = f[list(f.keys())[0]]
print(data.shape)

### Locate the labels of the ROIs

In [ ]:
def separate_tab(file_path):
    """
    A function to create a dictionary from a tab separated file
    
    """
    label_dict = {}
    with open(file_path) as ctab:
        reader = csv.reader(ctab, delimiter='\t')
        for row in reader:
            # do something with row
            label_dict[row[0].split()[0]] = row[0].split()[1]
    return label_dict

In [ ]:
ditt = {"a":[[1,2],[2,3],[3,4]], "b":2, "c":3}
print(ditt['a'][1][1])

In [ ]:
def count_roi_voxels_num(roi, label):
    return np.where(roi == label)[0].shape[0]

def generate_list_voxel_3d(roi, label):
    _axis_list = []
    all_3d = np.where(roi == label)
    for i in range(len(all_3d[0])):
        _axis_list.append([all_3d[0][i], all_3d[1][i], all_3d[2][i]])
    return _axis_list

def decompose_3d_to_voxel_id(combined_dict):
    # create an empty panda dataframe
    _df = pd.DataFrame(columns=['x', 'y', 'z', 'voxel_id', 'label'])
    counter = 0
    keys_dict = list(combined_dict.keys())
    for i in range(len(keys_dict)):
        for z in range(len(list(combined_dict[keys_dict[i]]))):
            # add a new row to the empty dataframe
            # use pd concat to add a new row
            single_row = pd.DataFrame([{'x': combined_dict[keys_dict[i]][z][0], 'y': combined_dict[keys_dict[i]][z][1], 'z': combined_dict[keys_dict[i]][z][2], 'voxel_id': counter, 'label': keys_dict[i]}])
            _df = pd.concat([_df, single_row], axis=0, ignore_index=True)
            counter += 1
    return _df

def concat_all_designs(design_file):
    # create an empty array
    _1d_list = []
    ## create strings to read data
    total_num = 0
    for i in range(37):
        for z in range(14):
            try:
                _filename = pjoin(design_file, 'design_session' + str(i+1).zfill(2)+'_run'+str(z+1).zfill(2)+'.tsv')
                ## read tsv data
                _data = pd.read_csv(_filename, sep='\t', header=None).values
                
                # get num larger than 0
                _data_nonzero = _data[_data > 0]
                _1d_list.append(_data_nonzero)
            except:
                continue
    return np.array(list(itertools.chain(*_1d_list)))
    

# def concat_all_betas(betas_file):
#     # create an 4d empty array
#     _4d_array = np.zeros((750*37, 83, 104, 81))
#     ## create strings to read data
#     for i in range(37):
#         _filename = pjoin(betas_file, 'betas_session' + str(i+1).zfill(2)+'.hdf5')
#     ## read data
#         f = h5py.File(_filename, 'r')
#         _data = f[list(f.keys())[0]]
#         _4d_array[i*750:(i+1)*750,:,:,:] = _data
#     return _4d_array


# def get_betas_to_3d(df_decompose, stimuli_design_list, stimuli_beta):
#     voxel_id_list = [i for i in range(len(df_decompose.voxel_id))]
#     pd_table_train = pd.DataFrame(columns=voxel_id_list)
#     pd_table_test = pd.DataFrame(columns=voxel_id_list)

#     unique_session_list = np.unique(stimuli_design_list)

#     for i in unique_session_list:
#         loc_betas_by_session_list = np.where(stimuli_design_list == i)[0]
#         single_beta_one_stimuli = []
#         for z in range(len(df_decompose.voxel_id)):
#             # get the xyz coordinate by voxel_id
#             _xyz = df_decompose[df_decompose['voxel_id'] == z][['x', 'y', 'z']].values[0]
#             # get the beta value by xyz coordinate
#             _beta = np.mean(stimuli_beta[loc_betas_by_session_list,_xyz[2],_xyz[1],_xyz[0]], axis=0)
#             single_beta_one_stimuli.append(_beta)
#         pd_table_train.loc[i] = single_beta_one_stimuli

#     return pd_table_train, pd_table_test

In [ ]:
roi_dir = pjoin(basedir, 'nsd_roi')

# prf rois
lh_prf_visual_file = pjoin(roi_dir, 'lh.prf-visualrois.nii.gz')
rh_prf_visual_file = pjoin(roi_dir, 'rh.prf-visualrois.nii.gz')

# import prf labels (ctab file)
prf_visual_labels_file = pjoin(roi_dir, 'prf-visualrois.mgz.ctab')

prf_visualrois_lables = separate_tab(prf_visual_labels_file)

# kastner rois
kastner_file = pjoin(roi_dir, 'Kastner2015.nii.gz')

kastner_labels_file = pjoin(roi_dir, 'Kastner2015.mgz.ctab')
kastner_labels = separate_tab(kastner_labels_file)
print(kastner_labels)

In [ ]:
design_file = pjoin(basedir, 'nsd_design')
lh_prf_img = nib.load(lh_prf_visual_file)
rh_prf_img = nib.load(rh_prf_visual_file)
kastner_img = nib.load(kastner_file)

lh_prf_data = lh_prf_img.get_fdata()
rh_prf_data = rh_prf_img.get_fdata()
kastner_data = kastner_img.get_fdata()

In [ ]:
# step 1, get roi size (how many voxels)

total_voxel_num = 0
for i in range(1,8):
    voxel_num_left = count_roi_voxels_num(lh_prf_data, int(i))
    voxel_num_right = count_roi_voxels_num(rh_prf_data, int(i))
    total_voxel_num += voxel_num_left + voxel_num_right

for i in range(8,12):
    voxel_num = count_roi_voxels_num(kastner_data, int(i))
    total_voxel_num += voxel_num

print("total voxel number: ", total_voxel_num)


In [ ]:
# step 2, label all the voxels and save it to a list

dict4combined_axis = {}
_tmp_dict = {}


for i in range(1,8):
    _tmp_dict["left_"+prf_visualrois_lables[str(i)]] = generate_list_voxel_3d(lh_prf_data, i)
    _tmp_dict["right_"+prf_visualrois_lables[str(i)]] = generate_list_voxel_3d(rh_prf_data, i)
    # integrate left and right
    left_right = [i for i in _tmp_dict["left_"+prf_visualrois_lables[str(i)]]]
    left_right.extend([i for i in _tmp_dict["right_"+prf_visualrois_lables[str(i)]]])
    dict4combined_axis[prf_visualrois_lables[str(i)]] = left_right


for i in range(8,12):
    dict4combined_axis[kastner_labels[str(i)]] = generate_list_voxel_3d(kastner_data, i)

print(dict4combined_axis.keys())

voxel_locator = decompose_3d_to_voxel_id(dict4combined_axis)
print(voxel_locator)

In [ ]:
### read the shared pics as the test, and the rest are the train

# step 2.1 loading shared pics and classify train and test:
shared_1000 = pd.read_csv(pjoin(basedir, 'shared1000.tsv'), header=None)

In [ ]:
# step 3, use the labels to generate betas tables
# concat all design files
all_design_image = concat_all_designs(design_file)

In [ ]:
# step 4, generate betas tables
pd_table_train = pd.DataFrame()
pd_table_test = pd.DataFrame()


stimuli_index = 0
for i in range(37):
    _filename = pjoin(betas_dir, 'betas_session' + str(i+1).zfill(2)+'.hdf5')
    with h5py.File(_filename, 'r') as f:
        # get data key
        data_key = list(f.keys())[0]
        dataset = f[data_key]
        for j in range(dataset.shape[0]):
            single_stimuli_voxels_array = np.zeros((len(voxel_locator.index), 1))
            for index, (x, y, z) in enumerate(zip(voxel_locator['z'], voxel_locator['y'], voxel_locator['x'])):
                single_stimuli_voxels_array[index] = dataset[j, x, y, z]
            _one_stimuli = pd.DataFrame(single_stimuli_voxels_array[:,0]/300, columns=[all_design_image[stimuli_index]])
            if all_design_image[stimuli_index] not in shared_1000.values:
                pd_table_train = pd.concat([pd_table_train, _one_stimuli], axis=1)
            else:
                pd_table_test = pd.concat([pd_table_test, _one_stimuli], axis=1)
            stimuli_index += 1

# add a label column to pd_table_train
pd_table_train['label'] = voxel_locator['label']
# add a label column to pd_table_test
pd_table_test['label'] = voxel_locator['label']

pd_table_train.to_csv(pjoin(basedir, 'train_betas.csv'))
pd_table_test.to_csv(pjoin(basedir, 'test_betas.csv'))